In [ ]:
import akshare as ak
import pandas as pd

In [ ]:
ag9999 = ak.spot_hist_sge(symbol='Ag99.99')
agm = ak.futures_hist_em(symbol="沪银主连", period="daily")

In [ ]:
df = pd.merge(ag9999, agm, left_on='date', right_on='时间', how='left')
df = df.rename(columns = {'close': 'ag9999', '收盘': 'agm'})
df = df[['date', 'ag9999', 'agm']]
df['date'] = pd.to_datetime(df['date'])
df['premium'] = df['ag9999'] - df['agm']
print(df['premium'].abs().describe())
print(df['premium'].abs().quantile(0.99))

In [ ]:
def calculate_drawdown_time(row, val):
     current_date = row['date']
     current_premium_abs = abs(row['premium'])
     if current_premium_abs <= val:
         return pd.Timedelta(0)
     subsequent_data = df[df['date'] > current_date]
     drawdown_index = subsequent_data[subsequent_data['premium'].abs() <= val].first_valid_index()
     if drawdown_index is not None:
         drawdown_date = df.loc[drawdown_index, 'date']
         return drawdown_date - current_date
     return pd.NaT
sell_point = 24

df[f'down_{sell_point}'] = df.apply(calculate_drawdown_time, args=(sell_point,), axis=1)

In [ ]:
threshold = 110
signal_df = df[(df['premium'] > threshold) | (df['premium'] < -1 * threshold)]
print(df['premium'].abs().quantile(0.95))
print(len(signal_df) * 100 / len(df))

signal_df

In [ ]:
df[625:665]